<a href="https://colab.research.google.com/github/Jmqcooper1/nlp1/blob/main/NLP1_2024_Practical_2_(student_version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

------
**You cannot apply any changes to this file, so please make sure to save it on your Google Colab drive or download it as a .ipynb file.**

------


# Practical 2: Representing Sentences with Neural Models

In this second practical, we will train neural network models to obtain sentence representations. We can then use these sentence representations for a downstream task such as sentiment classification.

In this notebook, we will help you to develop models for your experiments. But this time, next to completing the notebook, **you are expected to write a four-page scientific report with your findings**. Please still submit the notebook together with your scientific report so that we can reproduce your experiments. (Note: if you find it useful, you can split this notebook into multiple notebooks. If you do so, keep it mind that it should be possible for your TAs to reproduce the entire content of the notebooks without having to ask for clarifications or to copy and paste functions from one sub-notebook to another.) Make sure your code corresponds to the description you give in the report; we will deduct points if this is not the case. **Make sure you run all your code before submitting the notebook, and do not leave unnecessary print statements / cells in your notebook that are not intended for the grader.**

**Important!** The main purpose of this lab is for you to learn how to answer research questions by experimenting and then writing a scientific report.
So you will be *judged by the quality of your report* but will lose points if your experiments are not reproducible.
You can find the requirements for the report at the end of this notebook.


### Data set
We will use the [Stanford Sentiment Treebank](https://nlp.stanford.edu/sentiment/) (SST), which provides sentences, their binary tree structure, and fine-grained sentiment scores.
This dataset is different from the one we used in the first practical.
In Practical 1, a review consisted of several sentences, and we had one sentiment score for the whole review. Now, a review consists of a single sentence, and we have a sentiment score for each node in the binary tree that makes up the sentence, including the root node (i.e., we still have an overall sentiment score for the entire review). We will look at an example below.

In the first part of this practical we will only make use of the sentence tokens whereas in the second part we will also exploit the tree structure that is provided by the SST.

We will cover the following approaches:

- Bag-of-words (BOW)
- Continuous bag-of-words (CBOW)
- Deep continuous bag-of-words (Deep CBOW)
- LSTM
- Tree-LSTM


## Preparing the data

Let's first download the data set and take a look.

In [ ]:
import re
import random
import time
import math
import numpy as np
import nltk
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import logging
from pathlib import Path
import torch.cuda as cuda
plt.style.use('default')

In [3]:
!wget http://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip
!unzip trainDevTestTrees_PTB.zip

--2024-12-11 14:30:49--  http://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip [following]
--2024-12-11 14:30:49--  https://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 789539 (771K) [application/zip]
Saving to: ‘trainDevTestTrees_PTB.zip.2’

trainDevTestTrees_P 100%[===================>] 771.03K   198KB/s    in 3.9s    

2024-12-11 14:30:54 (198 KB/s) - ‘trainDevTestTrees_PTB.zip.2’ saved [789539/789539]

Archive:  trainDevTestTrees_PTB.zip
replace trees/dev.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [4]:
# this function reads in a textfile and fixes an issue with "\\"
def filereader(path):
  with open(path, mode="r", encoding="utf-8") as f:
    for line in f:
      yield line.strip().replace("\\","")

Let's look at a data point. It is a **flattened binary tree**, with sentiment scores at every node, and words as the leaves (or *terminal nodes*).

In [5]:
s = next(filereader("trees/dev.txt"))
print(s)

(3 (2 It) (4 (4 (2 's) (4 (3 (2 a) (4 (3 lovely) (2 film))) (3 (2 with) (4 (3 (3 lovely) (2 performances)) (2 (2 by) (2 (2 (2 Buy) (2 and)) (2 Accorsi))))))) (2 .)))


In [6]:
# We can use NLTK to better visualise the tree structure of the sentence
from nltk import Tree
from nltk.treeprettyprinter import TreePrettyPrinter
tree = Tree.fromstring(s)
print(TreePrettyPrinter(tree))

              3                                                                     
  ____________|____________________                                                  
 |                                 4                                                
 |        _________________________|______________________________________________   
 |       4                                                                        | 
 |    ___|______________                                                          |  
 |   |                  4                                                         | 
 |   |         _________|__________                                               |  
 |   |        |                    3                                              | 
 |   |        |               _____|______________________                        |  
 |   |        |              |                            4                       | 
 |   |        |              |            ________________|_

/tmp/ipykernel_18179/348436308.py:5: DeprecationWarning: 
    Class TreePrettyPrinter has been deprecated.  Import
    `TreePrettyPrinter` using `from nltk.tree import
    TreePrettyPrinter` instead.
  print(TreePrettyPrinter(tree))


The sentiment scores range from 0 (very negative) to 5 (very positive). Again, as you can see, every node in the tree is labeled with a sentiment score. For now, we will only use the score at the **root node**, i.e., the sentiment score for the complete sentence.

In [7]:
# Let's first make a function that extracts the tokens (the leaves).

def tokens_from_treestring(s):
  """extract the tokens from a sentiment tree"""
  return re.sub(r"\([0-9] |\)", "", s).split()

# let's try it on our example tree
tokens = tokens_from_treestring(s)
print(tokens)
print(len(tokens))

['It', "'s", 'a', 'lovely', 'film', 'with', 'lovely', 'performances', 'by', 'Buy', 'and', 'Accorsi', '.']
13


> *Warning: you could also parse a treestring using NLTK and ask it to return the leaves, but there seems to be an issue with NLTK not always correctly parsing the input, so do not rely on it.*

In [8]:
# We will also need the following function, but you can ignore this for now.
# It is explained later on.

SHIFT = 0
REDUCE = 1


def transitions_from_treestring(s):
  s = re.sub("\([0-5] ([^)]+)\)", "0", s)
  s = re.sub("\)", " )", s)
  s = re.sub("\([0-4] ", "", s)
  s = re.sub("\([0-4] ", "", s)
  s = re.sub("\)", "1", s)
  return list(map(int, s.split()))

<>:9: SyntaxWarning: invalid escape sequence '\('
<>:10: SyntaxWarning: invalid escape sequence '\)'
<>:11: SyntaxWarning: invalid escape sequence '\('
<>:12: SyntaxWarning: invalid escape sequence '\('
<>:13: SyntaxWarning: invalid escape sequence '\)'
<>:9: SyntaxWarning: invalid escape sequence '\('
<>:10: SyntaxWarning: invalid escape sequence '\)'
<>:11: SyntaxWarning: invalid escape sequence '\('
<>:12: SyntaxWarning: invalid escape sequence '\('
<>:13: SyntaxWarning: invalid escape sequence '\)'
/tmp/ipykernel_18179/1625531552.py:9: SyntaxWarning: invalid escape sequence '\('
  s = re.sub("\([0-5] ([^)]+)\)", "0", s)
/tmp/ipykernel_18179/1625531552.py:10: SyntaxWarning: invalid escape sequence '\)'
  s = re.sub("\)", " )", s)
/tmp/ipykernel_18179/1625531552.py:11: SyntaxWarning: invalid escape sequence '\('
  s = re.sub("\([0-4] ", "", s)
/tmp/ipykernel_18179/1625531552.py:12: SyntaxWarning: invalid escape sequence '\('
  s = re.sub("\([0-4] ", "", s)
/tmp/ipykernel_18179/162553

In [9]:
# Now let's first see how large our data sets are.
for path in ("trees/train.txt", "trees/dev.txt", "trees/test.txt"):
  print("{:16s} {:4d}".format(path, sum(1 for _ in filereader(path))))

trees/train.txt  8544
trees/dev.txt    1101
trees/test.txt   2210


You can see that the number of sentences is not very large. That's probably because the data set required so much manual annotation. However, it is large enough to train a neural network on.

It will be useful to store each data example in an `Example` object,
containing everything that we may need for each data point.
It will contain the tokens, the tree, the top-level sentiment label, and
the transitions (explained later).

In [10]:
from collections import namedtuple
from nltk import Tree

# A simple way to define a class is using namedtuple.
Example = namedtuple("Example", ["tokens", "tree", "label", "transitions"])


def examplereader(path, lower=False):
  """Returns all examples in a file one by one."""
  for line in filereader(path):
    line = line.lower() if lower else line
    tokens = tokens_from_treestring(line)
    tree = Tree.fromstring(line)  # use NLTK's Tree
    label = int(line[1])
    trans = transitions_from_treestring(line)
    yield Example(tokens=tokens, tree=tree, label=label, transitions=trans)


# Let's load the data into memory.
LOWER = False  # we will keep the original casing
train_data = list(examplereader("trees/train.txt", lower=LOWER))
dev_data = list(examplereader("trees/dev.txt", lower=LOWER))
test_data = list(examplereader("trees/test.txt", lower=LOWER))

print("train", len(train_data))
print("dev", len(dev_data))
print("test", len(test_data))

train 8544
dev 1101
test 2210


Let's check out an `Example` object.

In [11]:
example = dev_data[0]
print("First example:", example)
print("First example tokens:", example.tokens)
print("First example label:",  example.label)

First example: Example(tokens=['It', "'s", 'a', 'lovely', 'film', 'with', 'lovely', 'performances', 'by', 'Buy', 'and', 'Accorsi', '.'], tree=Tree('3', [Tree('2', ['It']), Tree('4', [Tree('4', [Tree('2', ["'s"]), Tree('4', [Tree('3', [Tree('2', ['a']), Tree('4', [Tree('3', ['lovely']), Tree('2', ['film'])])]), Tree('3', [Tree('2', ['with']), Tree('4', [Tree('3', [Tree('3', ['lovely']), Tree('2', ['performances'])]), Tree('2', [Tree('2', ['by']), Tree('2', [Tree('2', [Tree('2', ['Buy']), Tree('2', ['and'])]), Tree('2', ['Accorsi'])])])])])])]), Tree('2', ['.'])])]), label=3, transitions=[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1])
First example tokens: ['It', "'s", 'a', 'lovely', 'film', 'with', 'lovely', 'performances', 'by', 'Buy', 'and', 'Accorsi', '.']
First example label: 3


#### Vocabulary
A first step in most NLP tasks is collecting all the word types that appear in the data into a vocabulary, and counting the frequency of their occurrences. On the one hand, this will give us an overview of the word distribution of the data set (what are the most frequent words, how many rare words are there, ...). On the other hand, we will also use the vocabulary to map each word to a unique numeric ID, which is a more handy index than a string.

In [12]:
# Here we first define a class that can map a word to an ID (w2i)
# and back (i2w).

from collections import Counter, OrderedDict, defaultdict


class OrderedCounter(Counter, OrderedDict):
  """Counter that remembers the order elements are first seen"""
  def __repr__(self):
    return '%s(%r)' % (self.__class__.__name__,
                      OrderedDict(self))
  def __reduce__(self):
    return self.__class__, (OrderedDict(self),)


class Vocabulary:
  """A vocabulary, assigns IDs to tokens"""

  def __init__(self):
    self.freqs = OrderedCounter()
    self.w2i = {}
    self.i2w = []

  def count_token(self, t):
    self.freqs[t] += 1

  def add_token(self, t):
    self.w2i[t] = len(self.w2i)
    self.i2w.append(t)

  def build(self, min_freq=0):
    '''
    min_freq: minimum number of occurrences for a word to be included
              in the vocabulary
    '''
    self.add_token("<unk>")  # reserve 0 for <unk> (unknown words)
    self.add_token("<pad>")  # reserve 1 for <pad> (discussed later)

    tok_freq = list(self.freqs.items())
    tok_freq.sort(key=lambda x: x[1], reverse=True)
    for tok, freq in tok_freq:
      if freq >= min_freq:
        self.add_token(tok)

The vocabulary has by default an `<unk>` token and a `<pad>` token. The `<unk>` token is reserved for all words which do not appear in the training data (and for which, therefore, we cannot learn word representations). The function of the `<pad>` token will be explained later.


Let's build the vocabulary!


In [13]:
# This process should be deterministic and should have the same result
# if run multiple times on the same data set.

v = Vocabulary()
for data_set in (train_data,):
  for ex in data_set:
    for token in ex.tokens:
      v.count_token(token)

v.build()
print("Vocabulary size:", len(v.w2i))

Vocabulary size: 18280


Let's have a closer look at the properties of our vocabulary. Having a good idea of what it is like can facilitate data analysis and debugging later on.

In [14]:
# What is the ID for "century?"
v.w2i["century"]

1973

In [15]:
# What are the first 10 words in the vocabulary (based on their IDs)?
for i in range(10):
    print(v.i2w[i])

<unk>
<pad>
.
,
the
and
a
of
to
's


In [16]:
# What are the 10 most common words?
v.freqs.most_common(10)

[('.', 8024),
 (',', 7131),
 ('the', 6037),
 ('and', 4431),
 ('a', 4403),
 ('of', 4386),
 ('to', 2995),
 ("'s", 2544),
 ('is', 2536),
 ('that', 1915)]

In [17]:
# And how many words are there with frequency 1?
# (A fancy name for these is hapax legomena.)
list(v.freqs.values()).count(1)

9543

In [18]:
# Finally 20 random words from the vocabulary.
# This is a simple way to get a feeling for the data.
# You could use the `choice` function from the already imported `random` package
random.choices(v.i2w, k=20)

['fiercely',
 'tick-tock',
 'straining',
 'Internet',
 'swirling',
 'worked',
 'escapist',
 'underlying',
 'corniest',
 'shrewdly',
 'Jr.',
 'count',
 'avuncular',
 'turntablism',
 'regurgitates',
 'uncertainties',
 'cannibal',
 'One-sided',
 'Barrymore',
 'confession']

#### Sentiment label vocabulary

In [19]:
# Now let's map the sentiment labels 0-4 to a more readable form
i2t = ["very negative", "negative", "neutral", "positive", "very positive"]
print(i2t)
print(i2t[4])

['very negative', 'negative', 'neutral', 'positive', 'very positive']
very positive


In [20]:
# And let's also create the opposite mapping.
# We won't use a Vocabulary for this (although we could), since the labels
# are already numeric.
t2i = OrderedDict({p : i for p, i in zip(i2t, range(len(i2t)))})
print(t2i)
print(t2i['very positive'])

OrderedDict({'very negative': 0, 'negative': 1, 'neutral': 2, 'positive': 3, 'very positive': 4})
4


## PyTorch

We are going to need PyTorch and Google Colab does not have it installed by default. Run the cell below to install it.

*For installing PyTorch in your own computer, follow the instructions on [pytorch.org](pytorch.org) instead. This is for Google Colab only.*

In [21]:
import torch
print("Using torch", torch.__version__) # should say 1.7.0+cu101

Using torch 2.5.0


In [22]:
# Let's also import torch.nn, a PyTorch package that
# makes building neural networks more convenient.
from torch import nn

In [23]:
# PyTorch can run on CPU or on Nvidia GPU (video card) using CUDA
# This cell selects the GPU if one is available.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [24]:
# Seed manually to make runs reproducible
# You need to set this again if you do multiple runs of the same model
torch.manual_seed(42)

# When running on the CuDNN backend two further options must be set for reproducibility
if torch.cuda.is_available():
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

# BOW

Our first model is a rather simple neural **bag-of-words (BOW) model**.
Unlike the bag-of-words model that you used in the previous lab, where we would look at the presence / frequency of words in a text, here we associate each word with a multi-dimensional vector which expresses what sentiment is conveyed by the word. In particular, our BOW vectors will be of size 5, exactly our number of sentiment classes.

To classify a sentence, we **sum** the vectors of the words in the sentence and a bias vector. Because we sum the vectors, we lose word order: that's why we call this a neural bag-of-words model.

```
this   [0.0, 0.1, 0.1, 0.1, 0.0]
movie  [0.0, 0.1, 0.1, 0.2, 0.1]
is     [0.0, 0.1, 0.0, 0.0, 0.0]
stupid [0.9, 0.5, 0.1, 0.0, 0.0]

bias   [0.0, 0.0, 0.0, 0.0, 0.0]
--------------------------------
sum    [0.9, 0.8, 0.3, 0.3, 0.1]

argmax: 0 (very negative)
```

The **argmax** of this sum is our predicted label.

We initialize all vectors *randomly* and train them using cross-entropy loss.

#### Model class

In [25]:
class BOW(nn.Module):
  """A simple bag-of-words model"""

  def __init__(self, vocab_size, embedding_dim, vocab):
    super(BOW, self).__init__()
    self.vocab = vocab

    # this is a trainable look-up table with word embeddings
    self.embed = nn.Embedding(vocab_size, embedding_dim)

    # this is a trainable bias term
    self.bias = nn.Parameter(torch.zeros(embedding_dim), requires_grad=True)

  def forward(self, inputs):
    # this is the forward pass of the neural network
    # it applies a function to the input and returns the output

    # this looks up the embeddings for each word ID in inputs
    # the result is a sequence of word embeddings
    embeds = self.embed(inputs)

    # the output is the sum across the time dimension (1)
    # with the bias term added
    logits = embeds.sum(1) + self.bias

    return logits

In [26]:
# Let's create a model.
vocab_size = len(v.w2i)
n_classes = len(t2i)
bow_model = BOW(vocab_size, n_classes, v)
print(bow_model)

BOW(
  (embed): Embedding(18280, 5)
)


> **Hey, wait, where is the bias vector?**
> PyTorch does not print Parameters, only Modules!

> We can print it ourselves though, to check that it is there.

In [27]:

# Here we print each parameter name, shape, and if it is trainable.
def print_parameters(model):
  total = 0
  for name, p in model.named_parameters():
    total += np.prod(p.shape)
    print("{:24s} {:12s} requires_grad={}".format(name, str(list(p.shape)), p.requires_grad))
  print("\nTotal number of parameters: {}\n".format(total))


print_parameters(bow_model)

bias                     [5]          requires_grad=True
embed.weight             [18280, 5]   requires_grad=True

Total number of parameters: 91405



#### Preparing an example for input

To feed sentences to our PyTorch model, we need to convert a sequence of tokens to a sequence of IDs. The `prepare_example` function below takes care of this for us. We then use these IDs as indices for the word embedding table.

In [28]:
def prepare_example(example, vocab):
  """
  Map tokens to their IDs for a single example
  """

  # vocab returns 0 if the word is not there (i2w[0] = <unk>)
  x = [vocab.w2i.get(t, 0) for t in example.tokens]

  x = torch.LongTensor([x])
  x = x.to(device)

  y = torch.LongTensor([example.label])
  y = y.to(device)

  return x, y

In [29]:
x, y = prepare_example(dev_data[0], v)
print('x:', x)
print('y:', y)

x: tensor([[  28,    9,    6,  998,   16,   18,  998,  135,   32, 7688,    5,    0,
            2]], device='cuda:0')
y: tensor([3], device='cuda:0')


#### Evaluation
We now need to define an evaluation metric.
How many predictions do we get right? The accuracy will tell us.
Make sure that you understand this code block.


In [30]:
def simple_evaluate(model, data, prep_fn=prepare_example, **kwargs):
  """Accuracy of a model on given data set."""
  correct = 0
  total = 0
  model.eval()  # disable dropout (explained later)

  for example in data:

    # convert the example input and label to PyTorch tensors
    x, target = prep_fn(example, model.vocab)

    # forward pass without backpropagation (no_grad)
    # get the output from the neural network for input x
    with torch.no_grad():
      logits = model(x)

    # get the prediction
    prediction = logits.argmax(dim=-1)

    # add the number of correct predictions to the total correct
    correct += (prediction == target).sum().item()
    total += 1

  return correct, total, correct / float(total)

We are using accuracy as a handy evaluation metric. Please consider using [alternative metrics](https://scikit-learn.org/stable/modules/classes.html#classification-metrics) for your experiments if that makes more theoretical sense.

#### Example feed
For stochastic gradient descent (SGD) we will need a random training example for every update.
We implement this by shuffling the training data and returning examples one by one using `yield`.

Shuffling is optional so that we get to use this function to get validation and test examples, too.

In [31]:
def get_examples(data, shuffle=True, **kwargs):
  """Shuffle data set and return 1 example at a time (until nothing left)"""
  if shuffle:
    print("Shuffling training data")
    random.shuffle(data)  # shuffle training data each epoch
  for example in data:
    yield example

#### Exercise: Training function

Your task is now to complete the training loop below.
Before you do so, please read the section about optimisation.

**Optimisation**

As mentioned in the "Intro to PyTorch" notebook, one of the perks of using PyTorch is automatic differentiation. We will use it to train our BOW model.

We train our model by feeding it an input, performing a **forward** pass, obtaining an output prediction, and calculating a **loss** with our loss function.
After the gradients are computed in the **backward** pass, we can take a step on the surface of the loss function towards more optimal parameter settings (gradient descent).

The package we will use to do this optimisation is [torch.optim](https://pytorch.org/docs/stable/optim.html).

In [32]:
from torch import optim

Besides implementations of stochastic gradient descent (SGD), this package also implements the optimisation algorithm Adam, which we'll be using in this practical.
For the purposes of this assignment you do not need to know what Adam does besides that it uses gradient information to update our model parameters by calling:

```
optimizer.step()
```
Remember when we updated our parameters in the PyTorch tutorial in a loop?


```python
# update weights
learning_rate = 0.5
for f in net.parameters():
    # for each parameter, take a small step in the opposite dir of the gradient
    p.data = p.data - p.grad.data * learning_rate

```
The function call optimizer.step() does effectively the same thing.

*(If you want to know more about optimisation algorithms using gradient information, [this blog](http://ruder.io/optimizing-gradient-descent/.) gives a nice intuitive overview.)*

In [ ]:
def validate(model, data, eval_fn, batch_fn, prep_fn, eval_batch_size):
    """Helper function to evaluate the model."""
    model.eval()
    with torch.no_grad():
        val_loss, total, accuracy = eval_fn(
            model, data, 
            batch_size=eval_batch_size,
            batch_fn=batch_fn, 
            prep_fn=prep_fn
        )
    return val_loss, accuracy

def save_checkpoint(model, optimizer, path, best_eval, best_iter, iter_i, extra_data=None):
    """Helper function to save model checkpoint."""
    ckpt = {
        "state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "best_eval": best_eval,
        "best_iter": best_iter,
        "current_iter": iter_i
    }
    if extra_data:
        ckpt.update(extra_data)
    torch.save(ckpt, path)

In [ ]:
def train_model(model, optimizer, 
                num_iterations=10000,    # Original iteration limit
                print_every=500, 
                eval_every=500,
                batch_fn=get_examples,
                prep_fn=prepare_example,
                eval_fn=simple_evaluate,
                batch_size=32, 
                eval_batch_size=128,
                patience=5,              # Early stopping patience
                scheduler=None,          # Learning rate scheduler
                max_norm=1.0,           # Gradient clipping
                warmup_steps=100,       # Learning rate warmup
                accumulation_steps=4,    # Gradient accumulation
                checkpoint_every=1000):  # Periodic checkpoint saving
    """Train a model with optimized training process.
    
    Args:
        model: The neural network model
        optimizer: The optimizer (e.g., Adam)
        num_iterations: Maximum number of training iterations
        print_every: Steps between printing progress
        eval_every: Steps between evaluations
        batch_fn: Function to get batches of data
        prep_fn: Function to prepare examples
        eval_fn: Function to evaluate model
        batch_size: Training batch size
        eval_batch_size: Evaluation batch size
        patience: Early stopping patience
        scheduler: Learning rate scheduler
        max_norm: Maximum gradient norm
        warmup_steps: Number of warmup steps
        accumulation_steps: Number of steps for gradient accumulation
        checkpoint_every: Steps between checkpoints
    """
    try:
        # Initialize tracking variables
        iter_i = 0
        train_loss = 0.
        print_num = 0
        start = time.time()
        criterion = nn.CrossEntropyLoss()
        best_eval = 0.
        best_iter = 0
        no_improvement = 0

        # Store metrics for plotting
        losses = []
        accuracies = []
        val_losses = []
        lrs = []

        # Create checkpoint directory
        checkpoint_dir = Path("checkpoints")
        checkpoint_dir.mkdir(exist_ok=True)

        if eval_batch_size is None:
            eval_batch_size = batch_size

        # Get initial learning rate
        initial_lr = optimizer.param_groups[0]['lr']

        # Main training loop
        while True:  # when we run out of examples, shuffle and continue
            # Create progress bar for this epoch
            pbar = tqdm(batch_fn(train_data, batch_size=batch_size), 
                       desc=f'Iteration {iter_i}')
            
            for batch_idx, batch in enumerate(pbar):
                # forward pass
                model.train()
                x, targets = prep_fn(batch, model.vocab)
                
                # Move to GPU if available
                x = x.to(next(model.parameters()).device)
                targets = targets.to(next(model.parameters()).device)

                logits = model(x)

                # compute cross-entropy loss
                loss = criterion(logits.view([targets.size(0), -1]), targets.view(-1))
                # Scale loss for gradient accumulation
                loss = loss / accumulation_steps
                train_loss += loss.item() * accumulation_steps

                # backward pass
                loss.backward()

                # Gradient accumulation
                if (batch_idx + 1) % accumulation_steps == 0:
                    # Clip gradients
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
                    
                    # Learning rate warmup
                    if iter_i < warmup_steps:
                        lr_scale = min(1., float(iter_i + 1) / warmup_steps)
                        for pg in optimizer.param_groups:
                            pg['lr'] = lr_scale * initial_lr

                    # Update weights
                    optimizer.step()
                    optimizer.zero_grad()

                    # Track learning rate
                    lrs.append(optimizer.param_groups[0]['lr'])

                print_num += 1
                iter_i += 1

                # Update progress bar
                pbar.set_description(
                    f'Iter {iter_i}: loss={loss.item():.4f}, '
                    f'lr={optimizer.param_groups[0]["lr"]:.6f}'
                )

                # print info
                if iter_i % print_every == 0:
                    avg_loss = train_loss / print_num
                    print(f"\nIter {iter_i}: loss={avg_loss:.4f}, "
                          f"time={time.time()-start:.2f}s")
                    losses.append(avg_loss)
                    print_num = 0
                    train_loss = 0.

                # evaluate
                if iter_i % eval_every == 0:
                    # Validate model
                    val_loss, accuracy = validate(
                        model, dev_data, eval_fn, batch_fn, prep_fn, eval_batch_size
                    )
                    
                    accuracies.append(accuracy)
                    val_losses.append(val_loss)
                    print(f"iter {iter_i}: dev acc={accuracy:.4f}, val_loss={val_loss:.4f}")

                    # save best model parameters
                    if accuracy > best_eval:
                        print("new highscore")
                        best_eval = accuracy
                        best_iter = iter_i
                        no_improvement = 0
                        
                        # Save best model
                        save_checkpoint(
                            model, optimizer,
                            f"{checkpoint_dir}/{model.__class__.__name__}_best.pt",
                            best_eval, best_iter, iter_i
                        )
                    else:
                        no_improvement += 1

                    # Learning rate scheduling
                    if scheduler is not None:
                        scheduler.step(accuracy)

                    # Early stopping check
                    if no_improvement >= patience:
                        print(f"Early stopping triggered after {iter_i} iterations")
                        break

                # Periodic checkpoint saving
                if iter_i % checkpoint_every == 0:
                    save_checkpoint(
                        model, optimizer,
                        f"{checkpoint_dir}/{model.__class__.__name__}_{iter_i}.pt",
                        best_eval, best_iter, iter_i
                    )

                # Clear CUDA cache periodically
                if iter_i % 1000 == 0 and torch.cuda.is_available():
                    torch.cuda.empty_cache()

                # Check if we've hit iteration limit
                if iter_i >= num_iterations:
                    print("Reached maximum iterations")
                    break

            # Break outer loop if early stopping or max iterations
            if no_improvement >= patience or iter_i >= num_iterations:
                break

        # Final evaluation
        print("\nTraining finished! Loading best model...")
        best_model_path = f"{checkpoint_dir}/{model.__class__.__name__}_best.pt"
        ckpt = torch.load(best_model_path)
        model.load_state_dict(ckpt["state_dict"])

        # Evaluate on all sets
        model.eval()
        with torch.no_grad():
            train_loss, _, train_acc = eval_fn(
                model, train_data, batch_size=eval_batch_size,
                batch_fn=batch_fn, prep_fn=prep_fn)
            dev_loss, _, dev_acc = eval_fn(
                model, dev_data, batch_size=eval_batch_size,
                batch_fn=batch_fn, prep_fn=prep_fn)
            test_loss, _, test_acc = eval_fn(
                model, test_data, batch_size=eval_batch_size,
                batch_fn=batch_fn, prep_fn=prep_fn)

        print(f"Best model (iter {best_iter}): "
              f"train acc={train_acc:.4f}, dev acc={dev_acc:.4f}, "
              f"test acc={test_acc:.4f}")

        return {
            'losses': losses,
            'accuracies': accuracies,
            'val_losses': val_losses,
            'learning_rates': lrs,
            'best_accuracy': best_eval,
            'best_iteration': best_iter,
            'final_metrics': {
                'train_acc': train_acc,
                'dev_acc': dev_acc,
                'test_acc': test_acc,
                'train_loss': train_loss,
                'dev_loss': dev_loss,
                'test_loss': test_loss
            }
        }

    except Exception as e:
        print(f"Error during training: {str(e)}")
        # Save emergency checkpoint
        save_checkpoint(
            model, optimizer,
            f"{checkpoint_dir}/{model.__class__.__name__}_emergency.pt",
            best_eval, best_iter, iter_i,
            {'error': str(e)}
        )
        raise e

### Training the BOW model

In [ ]:
def train_with_multiple_seeds(model_class, model_args, seeds=[42, 123, 456], 
                            pretrained_vectors=None, freeze_embeddings=True,
                            **train_kwargs):
    """
    Train a model multiple times with different seeds.
    Handles both regular and pre-trained models.
    
    Args:
        model_class: The class of the model
        model_args: Dictionary of arguments for model initialization
        seeds: List of random seeds to use
        pretrained_vectors: Optional pre-trained word vectors
        freeze_embeddings: Whether to freeze embeddings (if pre-trained)
        **train_kwargs: Arguments for train_model function
    """
    all_stats = []
    
    print(f"Training with seeds: {seeds}")
    if pretrained_vectors is not None:
        print(f"Using pre-trained embeddings ({'frozen' if freeze_embeddings else 'trainable'})")
    
    
    for seed in seeds:
        print(f"\nTraining with seed {seed}")
        # Set seeds for reproducibility
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)
        
        # Initialize new model
        model = model_class(**model_args)
        
        # Initialize pre-trained embeddings if provided
        if pretrained_vectors is not None:
            with torch.no_grad():
                model.embed.weight.data.copy_(torch.from_numpy(pretrained_vectors))
                model.embed.weight.requires_grad = not freeze_embeddings
        
        model = model.to(device)
        
        # Initialize optimizer and scheduler
        optimizer = optim.AdamW(
            model.parameters(),
            lr=2e-4,
            weight_decay=0.01
        )
        
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='max',
            factor=0.5,
            patience=2,
            min_lr=1e-6
        )
        
        # Train model
        stats = train_model(
            model=model,
            optimizer=optimizer,
            scheduler=scheduler,
            **train_kwargs
        )
        
        all_stats.append(stats)
    
    # Calculate average and std dev
    test_accuracies = [s['final_metrics']['test_acc'] for s in all_stats]
    mean_test_acc = np.mean(test_accuracies)
    std_test_acc = np.std(test_accuracies)
    
    combined_stats = all_stats[0].copy()
    combined_stats['mean_test_acc'] = mean_test_acc
    combined_stats['std_test_acc'] = std_test_acc
    combined_stats['individual_runs'] = {
        f'seed_{seed}': stats for seed, stats in zip(seeds, all_stats)
    }
    
    print(f"\nFinal Results across {len(seeds)} seeds:")
    print(f"Test Accuracy: {mean_test_acc:.4f} ± {std_test_acc:.4f}")
    
    return combined_stats

In [ ]:
def plot_model_comparison(training_stats_list, model_names, figsize=(15, 12)):
    """Enhanced comparison plots including std dev and individual runs."""
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=figsize)
    colors = ['b', 'r', 'g', 'c', 'm']
    
    # Plot 1: Training Loss Comparison (using first run for clarity)
    for i, (stats, name) in enumerate(zip(training_stats_list, model_names)):
        ax1.plot(stats['losses'], f'{colors[i]}-', label=f'{name} Train')
        ax1.plot(stats['val_losses'], f'{colors[i]}--', label=f'{name} Val')
    ax1.set_title('Loss Comparison')
    ax1.set_xlabel('Evaluation Step')
    ax1.set_ylabel('Loss')
    ax1.grid(True)
    ax1.legend()
    
    # Plot 2: Accuracy Comparison with std dev bands
    for i, (stats, name) in enumerate(zip(training_stats_list, model_names)):
        # Plot mean accuracy
        runs = stats['individual_runs'].values()
        all_accs = np.array([run['accuracies'] for run in runs])
        mean_acc = np.mean(all_accs, axis=0)
        std_acc = np.std(all_accs, axis=0)
        
        x = np.arange(len(mean_acc))
        ax2.plot(x, mean_acc, f'{colors[i]}-', label=name)
        ax2.fill_between(x, mean_acc-std_acc, mean_acc+std_acc, 
                        color=colors[i], alpha=0.2)
    ax2.set_title('Validation Accuracy (with ±1 std)')
    ax2.set_xlabel('Evaluation Step')
    ax2.set_ylabel('Accuracy')
    ax2.grid(True)
    ax2.legend()
    
    # Plot 3: Learning Rate Comparison
    for i, (stats, name) in enumerate(zip(training_stats_list, model_names)):
        ax3.plot(stats['learning_rates'], f'{colors[i]}-', label=name)
    ax3.set_title('Learning Rate Evolution')
    ax3.set_xlabel('Training Step')
    ax3.set_ylabel('Learning Rate')
    ax3.set_yscale('log')
    ax3.grid(True)
    ax3.legend()
    
    # Plot 4: Final Performance Box Plot
    splits = ['Train', 'Dev', 'Test']
    x = np.arange(len(splits))
    width = 0.8 / len(model_names)
    
    for i, (stats, name) in enumerate(zip(training_stats_list, model_names)):
        position = x + (i * width) - (width * len(model_names)/2) + width/2
        
        # Collect accuracies across runs
        run_accs = np.array([[run['final_metrics'][f'{split.lower()}_acc'] 
                             for split in splits]
                            for run in stats['individual_runs'].values()])
        
        means = np.mean(run_accs, axis=0)
        stds = np.std(run_accs, axis=0)
        
        # Plot bars with error bars
        bars = ax4.bar(position, means, width, label=name)
        ax4.errorbar(position, means, yerr=stds, fmt='none', 
                    color='black', capsize=5)
        
        # Add text annotations
        for j, (mean, std) in enumerate(zip(means, stds)):
            ax4.text(position[j], mean, f'{mean:.3f}\n±{std:.3f}',
                    ha='center', va='bottom', rotation=90)
    
    ax4.set_title('Final Performance (mean ± std)')
    ax4.set_ylabel('Accuracy')
    ax4.set_xticks(x)
    ax4.set_xticklabels(splits)
    ax4.legend()
    ax4.grid(True, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed statistics
    print("\nDetailed Model Comparison:")
    for name, stats in zip(model_names, training_stats_list):
        print(f"\n{name}:")
        print(f"Test Accuracy: {stats['mean_test_acc']:.4f} ± {stats['std_test_acc']:.4f}")
        print("Individual run test accuracies:", 
              [f"{run['final_metrics']['test_acc']:.4f}" 
               for run in stats['individual_runs'].values()])

def plot_smoothed_comparison(training_stats_list, model_names, window=5):
    """Enhanced smoothed comparison with confidence bands."""
    plt.figure(figsize=(15, 6))
    
    for i, (stats, name) in enumerate(zip(training_stats_list, model_names)):
        runs = stats['individual_runs'].values()
        all_accs = np.array([pd.Series(run['accuracies']).rolling(window).mean()
                            for run in runs])
        
        mean_acc = np.nanmean(all_accs, axis=0)
        std_acc = np.nanstd(all_accs, axis=0)
        
        plt.plot(mean_acc, label=name)
        plt.fill_between(range(len(mean_acc)), 
                        mean_acc-std_acc, mean_acc+std_acc,
                        alpha=0.2)
    
    plt.title('Smoothed Validation Accuracy (with ±1 std)')
    plt.xlabel('Evaluation Step')
    plt.ylabel('Accuracy')
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:
# If everything is in place we can now train our first model!
bow_args = {
    'vocab_size': len(v.w2i),
    'num_classes': 5,
    'vocab': v
}

bow_training_stats = train_with_multiple_seeds(
    model_class=BOW,
    model_args=bow_args,
    seeds=[42, 123, 456],
    num_iterations=30000,
    print_every=500,
    eval_every=500,
    batch_size=32,
    eval_batch_size=128,
    patience=5,
    max_norm=1.0,
    warmup_steps=100,
    accumulation_steps=4,
    checkpoint_every=1000
)

BOW(
  (embed): Embedding(18280, 5)
)
Shuffling training data
Iter 1000: loss=5210.8817, time=0.57s
iter 1000: dev acc=0.2171
new highscore
Iter 2000: loss=4734.5682, time=1.21s
iter 2000: dev acc=0.2162
Iter 3000: loss=4805.0908, time=1.88s
iter 3000: dev acc=0.2243
new highscore
Iter 4000: loss=4457.4811, time=2.51s
iter 4000: dev acc=0.2380
new highscore
Iter 5000: loss=4580.1939, time=3.11s
iter 5000: dev acc=0.2416
new highscore
Iter 6000: loss=4405.2550, time=3.72s
iter 6000: dev acc=0.2425
new highscore
Iter 7000: loss=4216.9528, time=4.33s
iter 7000: dev acc=0.2434
new highscore
Iter 8000: loss=4169.1576, time=4.91s
iter 8000: dev acc=0.2461
new highscore
Shuffling training data
Iter 9000: loss=4180.1943, time=5.50s
iter 9000: dev acc=0.2461
Iter 10000: loss=3939.1272, time=6.09s
iter 10000: dev acc=0.2516
new highscore
Iter 11000: loss=3993.5804, time=6.68s
iter 11000: dev acc=0.2534
new highscore
Iter 12000: loss=3822.1498, time=7.29s
iter 12000: dev acc=0.2598
new highscore


/tmp/ipykernel_18179/858537470.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(path)


best model iter 29000: train acc=0.3206, dev acc=0.2861, test acc=0.2738


Please note that we set the number of iterations to 30K as an indicative value, after which we simply stop training without checking for convergence. You should choose an appropriate number of iterations and motivate your decision. **This holds for all pre-set numbers of iteration in the following code blocks.**

# CBOW

We now continue with a **continuous bag-of-words (CBOW)** model. (*This is not the same as the word2vec CBOW model!*)

It is similar to the BOW model above, but now embeddings can have a dimension of *arbitrary size*.
This means that we can choose a higher dimensionality and learn more aspects of each word. We will still sum word vectors to get a sentence representation, but now the size of the resulting vector will no longer correspond to the number of sentiment classes.

So to turn the size of our summed vector into the number of output classes, we can *learn* a parameter matrix $W$ and multiply it by the sum vector $x$: $$Wx$$
If the size of $x$ is `d x 1`, we can set $W$ to be `5 x d`, so that the output of the matrix multiplication will be the of the desired size, `5 x 1`. Then, just like for the BOW model, we can obtain a prediction using the argmax function.

## Exercise: implement and train the CBOW model

Write a class `CBOW` that:

- has word embeddings with size 300
- sums the word vectors for the input words (just like in `BOW`)
- projects the resulting vector down to 5 units using a linear layer and a bias term (check out `nn.Linear`)

Train your CBOW model and plot the validation accuracy and training loss over time.

In [38]:
# YOUR CODE HERE
class CBOW(nn.Module):
    def __init__(self, vocab_size, num_classes, vocab, embedding_dim=300):
        super(CBOW, self).__init__()
        self.vocab = vocab
        
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        
        self.linear = nn.Linear(embedding_dim, num_classes, bias=True)

    def forward(self, inputs):
        embeds = self.embed(inputs)
        summed = embeds.sum(1)
        
        logits = self.linear(summed)
        
        return logits

In [ ]:
cbow_args = {
    'vocab_size': len(v.w2i),
    'num_classes': 5,
    'vocab': v,
    'embedding_dim': 300,
}

cbow_training_stats = train_with_multiple_seeds(
    model_class=CBOW,
    model_args=cbow_args,
    seeds=[42, 123, 456],
    num_iterations=30000,
    print_every=500,
    eval_every=500,
    batch_size=32,
    eval_batch_size=128,
    patience=5,
    max_norm=1.0,
    warmup_steps=100,
    accumulation_steps=4,
    checkpoint_every=1000
)

CBOW(
  (embed): Embedding(18280, 300)
  (linear): Linear(in_features=300, out_features=5, bias=True)
)
Shuffling training data
Iter 1000: loss=2677.7609, time=1.77s
iter 1000: dev acc=0.2489
new highscore
Iter 2000: loss=2305.0685, time=3.62s
iter 2000: dev acc=0.2788
new highscore
Iter 3000: loss=2064.4200, time=5.64s
iter 3000: dev acc=0.2725
Iter 4000: loss=1963.8503, time=7.48s
iter 4000: dev acc=0.2979
new highscore
Iter 5000: loss=1979.3512, time=9.48s
iter 5000: dev acc=0.2852
Iter 6000: loss=1885.8083, time=11.34s
iter 6000: dev acc=0.2861
Iter 7000: loss=1875.4106, time=13.21s
iter 7000: dev acc=0.2979
Iter 8000: loss=1899.4583, time=15.04s
iter 8000: dev acc=0.2834
Shuffling training data
Iter 9000: loss=1684.6027, time=16.98s
iter 9000: dev acc=0.2707
Iter 10000: loss=1617.6152, time=18.74s
iter 10000: dev acc=0.2797
Iter 11000: loss=1608.4602, time=20.55s
iter 11000: dev acc=0.3034
new highscore
Iter 12000: loss=1601.9276, time=22.40s
iter 12000: dev acc=0.3143
new highsco

/tmp/ipykernel_18179/858537470.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(path)


best model iter 26000: train acc=0.6696, dev acc=0.3579, test acc=0.3584


# Deep CBOW

To see if we can squeeze some more performance out of the CBOW model, we can make it deeper and non-linear by adding more layers and, e.g., tanh-activations.
By using more parameters we can learn more aspects of the data, and by using more layers and non-linearities, we can try to learn a more complex function.
This is not something that always works. If the input-output mapping of your data is simple, then a complicated function could easily overfit on your training set, thereby leading to poor generalization.

#### Exercise: write Deep CBOW class and train it

Write a class `DeepCBOW`.

In your code, make sure that your `output_layer` consists of the following:
- A linear transformation from E units to D units.
- A Tanh activation
- A linear transformation from D units to D units
- A Tanh activation
- A linear transformation from D units to 5 units (our output classes).

E is the size of the word embeddings (please use E=300) and D for the size of a hidden layer (please use D=100).

We recommend using [nn.Sequential](https://pytorch.org/docs/stable/nn.html?highlight=sequential#torch.nn.Sequential) to implement this exercise.

In [41]:
# YOUR CODE HERE
class DeepCBOW(nn.Module):
    def __init__(self, vocab_size, num_classes, vocab, embedding_dim=300, hidden_dim=100):
        super(DeepCBOW, self).__init__()
        self.vocab = vocab
        
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        
        self.output_layer = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),  # 300 -> 100 | E -> D
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),     # 100 -> 100
            nn.Tanh(),
            nn.Linear(hidden_dim, num_classes)     # 100 -> 5 | D -> C
        )

    def forward(self, inputs):
        embeds = self.embed(inputs)
        summed = embeds.sum(1)
        
        logits = self.output_layer(summed)
        
        return logits

In [ ]:
deepcbow_args = {
    'vocab_size': len(v.w2i),
    'num_classes': 5,
    'vocab': v,
    'embedding_dim': 300,
    'hidden_dim': 100,
}

deepcbow_training_stats = train_with_multiple_seeds(
    model_class=DeepCBOW,
    model_args=deepcbow_args,
    seeds=[42, 123, 456],
    num_iterations=30000,
    print_every=500,
    eval_every=500,
    batch_size=32,
    eval_batch_size=128,
    patience=5,
    max_norm=1.0,
    warmup_steps=100,
    accumulation_steps=4,
    checkpoint_every=1000
)

DeepCBOW(
  (embed): Embedding(18280, 300)
  (output_layer): Sequential(
    (0): Linear(in_features=300, out_features=100, bias=True)
    (1): Tanh()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): Tanh()
    (4): Linear(in_features=100, out_features=5, bias=True)
  )
)
Shuffling training data
Iter 1000: loss=1581.4196, time=1.89s
iter 1000: dev acc=0.2779
new highscore
Iter 2000: loss=1587.4563, time=4.07s
iter 2000: dev acc=0.2979
new highscore
Iter 3000: loss=1565.6903, time=6.13s
iter 3000: dev acc=0.3052
new highscore
Iter 4000: loss=1546.8563, time=8.22s
iter 4000: dev acc=0.2770
Iter 5000: loss=1526.0611, time=10.16s
iter 5000: dev acc=0.3006
Iter 6000: loss=1530.8873, time=12.19s
iter 6000: dev acc=0.3415
new highscore
Iter 7000: loss=1513.1408, time=14.26s
iter 7000: dev acc=0.3224
Iter 8000: loss=1472.3755, time=16.32s
iter 8000: dev acc=0.3306
Shuffling training data
Iter 9000: loss=1459.2616, time=18.34s
iter 9000: dev acc=0.3479
new highscore
Iter 1

/tmp/ipykernel_18179/858537470.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(path)


best model iter 27000: train acc=0.6413, dev acc=0.3688, test acc=0.3629


# Pre-trained word embeddings

The Stanford Sentiment Treebank is a rather small data set, since it required fine-grained manual annotatation. This makes it difficult for the Deep CBOW model to learn good word embeddings, i.e. to learn good word representations for the words in our vocabulary.
In fact, the only error signal that the network receives is from predicting the sentiment of entire sentences!

To start off with better word representations, we can download **pre-trained word embeddings**.
You can choose which pre-trained word embeddings to use:

- **GloVe**. The "original" Stanford Sentiment classification [paper](http://aclweb.org/anthology/P/P15/P15-1150.pdf) used Glove embeddings, which are just another method (like *word2vec*) to get word embeddings from unannotated text. Glove is described in the following paper which you should cite if you use them:
> Jeffrey Pennington, Richard Socher, and Christopher Manning. ["Glove: Global vectors for word representation."](https://nlp.stanford.edu/pubs/glove.pdf) EMNLP 2014.

- **Word2Vec**. This is the method that you learned about in class, described in:
> Mikolov, Tomas, et al. ["Distributed representations of words and phrases and their compositionality."](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) Advances in neural information processing systems. 2013.

Using these pre-trained word embeddings, we can initialize our word embedding lookup table and start form a point where similar words are already close to one another in the distributional semantic space.

You can choose to keep the word embeddings **fixed** or to train them further, specialising them to the task at hand.
We will keep them fixed for now.

For the purposes of this lab, it is enough if you understand how word2vec works (whichever vectors you use), but if you are interested, we encourage you to also check out the GloVe paper.

You can either download the word2vec vectors, or the Glove vectors.
If you want to compare your results to the Stanford paper later on, then you should use Glove.
**At the end of this lab you have the option to compare which vectors give you the best performance. For now, simply choose one of them and continue with that.**

[**OPTIONAL in case you don't want to mount Google Drive:** instead of running all the 5 boxes below, you can 1) download the GloVe and word2vec in your local machine, 2) upload them on your Drive folder ("My Drive"). Then, uncomment the first 2 lines in box 6 before writing your code!]


In [44]:
# This downloads the Glove 840B 300d embeddings.
# The original file is at http://nlp.stanford.edu/data/glove.840B.300d.zip
# Since that file is 2GB, we provide you with a *filtered version*
# which contains all the words you need for this data set.

# You only need to do this once.
# Please comment this cell out after downloading.

# !wget https://gist.githubusercontent.com/bastings/b094de2813da58056a05e8e7950d4ad1/raw/3fbd3976199c2b88de2ae62afc0ecc6f15e6f7ce/glove.840B.300d.sst.txt

In [45]:
# This downloads the word2vec 300D Google News vectors
# The file has been truncated to only contain words that appear in our data set.
# You can find the original file here: https://code.google.com/archive/p/word2vec/

# You only need to do this once.
# Please comment this out after downloading.
# !wget https://gist.githubusercontent.com/bastings/4d1c346c68969b95f2c34cfbc00ba0a0/raw/76b4fefc9ef635a79d0d8002522543bc53ca2683/googlenews.word2vec.300d.txt

In [46]:
# Mount Google Drive (to save the downloaded files)
from google.colab import drive
drive.mount('/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Copy word vectors *to* Google Drive

# You only need to do this once.
# Please comment this out after running it.
# !cp "glove.840B.300d.sst.txt" "/gdrive/My Drive/"
# !cp "googlenews.word2vec.300d.txt" "/gdrive/My Drive/"

In [ ]:
# If you copied the word vectors to your Drive before,
# here is where you copy them back to the Colab notebook.

# Copy Glove vectors *from* Google Drive
!cp "/gdrive/My Drive/glove.840B.300d.sst.txt" .
!cp "/gdrive/My Drive/googlenews.word2vec.300d.txt" .

In [47]:
# Uncomment these 2 lines below if went for the OPTIONAL method described above
!cp "glove.840B.300d.sst.txt" "./"
!cp "googlenews.word2vec.300d.txt" "./"

cp: 'glove.840B.300d.sst.txt' and './glove.840B.300d.sst.txt' are the same file
cp: 'googlenews.word2vec.300d.txt' and './googlenews.word2vec.300d.txt' are the same file


At this point you have the pre-trained word embedding files, but what do they look like?

In [48]:
# Exercise: Print the first 4 lines of the files that you downloaded.
# What do you see?
import os
if os.path.exists("glove.840B.300d.sst.txt"):
    file_path_glove = "glove.840B.300d.sst.txt"
    file_path_w2v = "googlenews.word2vec.300d.txt"
else:
    file_path_glove = "/gdrive/My Drive/glove.840B.300d.sst.txt"
    file_path_w2v = "/gdrive/My Drive/googlenews.word2vec.300d.txt"

# Print first 4 lines of GloVe embeddings
print("GloVe first 4 lines:")
reader = filereader(file_path_glove)
for i in range(4):
    line = next(reader)
    print(line)

print("\nWord2Vec first 4 lines:")
# Print first 4 lines of Word2Vec embeddings
reader = filereader(file_path_w2v)
for i in range(4):
    line = next(reader)
    print(line)

GloVe first 4 lines:
, -0.082752 0.67204 -0.14987 -0.064983 0.056491 0.40228 0.0027747 -0.3311 -0.30691 2.0817 0.031819 0.013643 0.30265 0.0071297 -0.5819 -0.2774 -0.062254 1.1451 -0.24232 0.1235 -0.12243 0.33152 -0.006162 -0.30541 -0.13057 -0.054601 0.037083 -0.070552 0.5893 -0.30385 0.2898 -0.14653 -0.27052 0.37161 0.32031 -0.29125 0.0052483 -0.13212 -0.052736 0.087349 -0.26668 -0.16897 0.015162 -0.0083746 -0.14871 0.23413 -0.20719 -0.091386 0.40075 -0.17223 0.18145 0.37586 -0.28682 0.37289 -0.16185 0.18008 0.3032 -0.13216 0.18352 0.095759 0.094916 0.008289 0.11761 0.34046 0.03677 -0.29077 0.058303 -0.027814 0.082941 0.1862 -0.031494 0.27985 -0.074412 -0.13762 -0.21866 0.18138 0.040855 -0.113 0.24107 0.3657 -0.27525 -0.05684 0.34872 0.011884 0.14517 -0.71395 0.48497 0.14807 0.62287 0.20599 0.58379 -0.13438 0.40207 0.18311 0.28021 -0.42349 -0.25626 0.17715 -0.54095 0.16596 -0.036058 0.08499 -0.64989 0.075549 -0.28831 0.40626 -0.2802 0.094062 0.32406 0.28437 -0.26341 0.11553 0.071918 -

#### Exercise: New Vocabulary

Since we now use pre-trained word embeddings, we need to create a new vocabulary.
This is because of two reasons:

1. We do not have pre-trained word embeddings for all words in our SST training set, and we do not want words in our vocabulary for which we have no word embeddings.
2. We should be able to look up the pre-trained word embedding for words in the validation and test set, even if these words are unseen in training.

Now, create a new vocabulary object `v` based on the word set of pre-trained embeddings, and load the corresponding embeddings into a list `vectors`.

The vocabulary `v` should consist of:
 - a  `<unk>` token at position 0,
 - a  `<pad>` token at position 1,
 - and then all words in the pre-trained embedding set.


After storing each vector in a list `vectors`, turn the list into a numpy matrix like this:
```python
 vectors = np.stack(vectors, axis=0)
```

Remember to add new embeddings for the `<unk>` and `<pad>` tokens, as they're not part of the word2vec/GloVe embeddings. These embeddings can be randomly initialized or 0-valued, think about what makes sense and see what the effects are.


In [49]:
# YOUR CODE HERE
v = Vocabulary()

v.add_token("<unk>") 
v.add_token("<pad>") 

embedding_dim = 300
vectors = [
    np.zeros(embedding_dim),  
    np.zeros(embedding_dim)   
]

for line in filereader(file_path_glove):
    parts = line.split()
    word = parts[0]
    vector = np.array([float(x) for x in parts[1:]])
    
    v.add_token(word)
    vectors.append(vector)

vectors = np.stack(vectors, axis=0)

#### Exercise: words not in our pre-trained set

How many words in the training, dev, and test set are also in your vector set?
How many words are not there?

Store the words that are not in the word vector set in the set below.

In [50]:
words_not_found = set()
# YOUR CODE HERE
for dataset in (train_data, dev_data, test_data):
    words_in_set = 0
    words_total = 0
    
    for example in dataset:
        for token in example.tokens:
            words_total += 1
            if v.w2i.get(token, 0) == 0:  
                words_not_found.add(token)
            else:
                words_in_set += 1
                
    print(f"Words in pre-trained set: {words_in_set}/{words_total}")
    print(f"Coverage: {words_in_set/words_total:.2%}")

print(f"\nTotal unique unknown words: {len(words_not_found)}")


Words in pre-trained set: 162125/163566
Coverage: 99.12%
Words in pre-trained set: 21066/21274
Coverage: 99.02%
Words in pre-trained set: 42043/42405
Coverage: 99.15%

Total unique unknown words: 976


#### Exercise: train Deep CBOW with (fixed) pre-trained embeddings

Now train Deep CBOW again using the pre-trained word vectors.


In [51]:
# We define a new class that inherits from DeepCBOW.
class PTDeepCBOW(DeepCBOW):
    def __init__(self, vocab_size, output_dim, vocab, embedding_dim=300, hidden_dim=100):
        super(PTDeepCBOW, self).__init__(
            vocab_size=vocab_size,
            num_classes=output_dim,  
            vocab=vocab,
            embedding_dim=embedding_dim,
            hidden_dim=hidden_dim
        )

In [ ]:
pt_deep_cbow_args = {
    'vocab_size': len(v.w2i),
    'embedding_dim': 300,
    'hidden_dim': 100,
    'output_dim': 5,
    'vocab': v
}

pt_deep_cbow_training_stats = train_with_multiple_seeds(
    model_class=PTDeepCBOW,
    model_args=pt_deep_cbow_args,
    pretrained_vectors=vectors,
    freeze_embeddings=True,  # Set to False if you want to fine-tune embeddings
    seeds=[42, 123, 456],
    num_iterations=30000,
    print_every=500,
    eval_every=500,
    batch_size=32,
    eval_batch_size=128,
    patience=5,
    max_norm=1.0,
    warmup_steps=100,
    accumulation_steps=4,
    checkpoint_every=1000
)

PTDeepCBOW(
  (embed): Embedding(20727, 300)
  (output_layer): Sequential(
    (0): Linear(in_features=300, out_features=100, bias=True)
    (1): Tanh()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): Tanh()
    (4): Linear(in_features=100, out_features=5, bias=True)
  )
)
Shuffling training data
Iter 1000: loss=1482.5307, time=0.66s
iter 1000: dev acc=0.3887
new highscore
Iter 2000: loss=1391.5471, time=1.54s
iter 2000: dev acc=0.3651
Iter 3000: loss=1332.0316, time=2.37s
iter 3000: dev acc=0.3987
new highscore
Iter 4000: loss=1389.1251, time=3.20s
iter 4000: dev acc=0.3833
Iter 5000: loss=1350.4859, time=4.03s
iter 5000: dev acc=0.4142
new highscore
Iter 6000: loss=1335.8725, time=4.84s
iter 6000: dev acc=0.3987
Iter 7000: loss=1350.3854, time=5.70s
iter 7000: dev acc=0.4033
Iter 8000: loss=1329.1784, time=6.53s
iter 8000: dev acc=0.4005
Shuffling training data
Iter 9000: loss=1329.4969, time=7.32s
iter 9000: dev acc=0.3760
Iter 10000: loss=1299.8670, time=8.14

/tmp/ipykernel_18179/858537470.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(path)


best model iter 19000: train acc=0.4611, dev acc=0.4423, test acc=0.4452


**It looks like we've hit what is possible with just using words.**
Let's move on by incorporating word order!

# LSTM

It is time to get more serious. Even with pre-trained word embeddings and multiple layers, we still seem to do pretty badly at sentiment classification.
The next step we can take is to introduce word order again, dropping our independence assumptions. In this way, we can get a representation of the sentence as an ordered set of tokens.

We will get this representation using a **Long Short-Term Memory** (LSTM). As an exercise, we will code our own LSTM cell, so that we get comfortable with its inner workings.
Once we have an LSTM cell, we can call it repeatedly, updating its hidden state one word at a time:

```python
rnn = MyLSTMCell(input_size, hidden_size)

hx = torch.zeros(1, hidden_size)  # initial hidden state
cx = torch.zeros(1, hidden_size)  # initial memory cell
output = []                       # to save intermediate LSTM states

# feed one word at a time
for i in range(n_timesteps):
  hx, cx = rnn(input[i], (hx, cx))
  output.append(hx)
```

If you need some more help understanding LSTMs, you can check out these resources:
- Blog post (highly recommended): http://colah.github.io/posts/2015-08-Understanding-LSTMs/
- Paper covering LSTM formulas in detail: https://arxiv.org/abs/1503.04069


#### Exercise: Finish the LSTM cell below.
You will need to implement the LSTM formulas:

$$
\begin{array}{ll}
        i = \sigma(W_{ii} x + b_{ii} + W_{hi} h + b_{hi}) \\
        f = \sigma(W_{if} x + b_{if} + W_{hf} h + b_{hf}) \\
        g = \tanh(W_{ig} x + b_{ig} + W_{hg} h + b_{hg}) \\
        o = \sigma(W_{io} x + b_{io} + W_{ho} h + b_{ho}) \\
        c' = f * c + i * g \\
        h' = o \tanh(c') \\
\end{array}
 $$

where $\sigma$ is the sigmoid function.

*Note that the LSTM formulas can differ slightly between different papers. We use the PyTorch LSTM formulation here.*

In [55]:
class MyLSTMCell(nn.Module):
    """Our own LSTM cell"""

    def __init__(self, input_size, hidden_size, bias=True):
        """Creates the weights for this LSTM"""
        super(MyLSTMCell, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias

        # YOUR CODE HERE
        # Create the input-hidden weights (W_ii|W_if|W_ig|W_io)
        self.weight_ih = nn.Parameter(
            torch.empty(4 * hidden_size, input_size))
        
        # Create the hidden-hidden weights (W_hi|W_hf|W_hg|W_ho)
        self.weight_hh = nn.Parameter(
            torch.empty(4 * hidden_size, hidden_size))
        
        if bias:
            # Create input-hidden bias (b_ii|b_if|b_ig|b_io)
            self.bias_ih = nn.Parameter(torch.empty(4 * hidden_size))
            # Create hidden-hidden bias (b_hi|b_hf|b_hg|b_ho)
            self.bias_hh = nn.Parameter(torch.empty(4 * hidden_size))
        else:
            self.register_parameter('bias_ih', None)
            self.register_parameter('bias_hh', None)
        
        
        self.reset_parameters()

    def reset_parameters(self):
        """This is PyTorch's default initialization method"""
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)

    def forward(self, input_, hx, mask=None):
        """
    input is (batch, input_size)
    hx is ((batch, hidden_size), (batch, hidden_size))
    """
        prev_h, prev_c = hx

        # project input and prev state
        # YOUR CODE HERE
        gates = (torch.mm(input_, self.weight_ih.t()) + 
                torch.mm(prev_h, self.weight_hh.t()))
        
        if self.bias:
            gates = gates + self.bias_ih + self.bias_hh
            
        gates = gates.chunk(4, 1)

        # main LSTM computation
        i = torch.sigmoid(gates[0])  # input gate
        f = torch.sigmoid(gates[1])  # forget gate
        g = torch.tanh(gates[2])     # cell gate
        o = torch.sigmoid(gates[3])  # output gate
        
        # Update cell state and hidden state
        c = f * prev_c + i * g
        h = o * torch.tanh(c)

        return h, c

    def __repr__(self):
        return "{}({:d}, {:d})".format(
        self.__class__.__name__, self.input_size, self.hidden_size)

#### Optional: Efficient Matrix Multiplication

It is more efficient to do a few big matrix multiplications than to do many smaller ones. So we will implement the above cell using just **two** linear layers.

This is possible because the eight linear transformations contained in one forward pass through an LSTM cell can be reduced to just two:
$$W_h h + b_h$$
$$W_i x + b_i $$

with $h = $ `prev_h` and $x = $ `input_`.

and where:

$W_h =  \begin{pmatrix}
W_{hi}\\
W_{hf}\\
W_{hg}\\
W_{ho}
\end{pmatrix}$, $b_h = \begin{pmatrix}
b_{hi}\\
b_{hf}\\
b_{hg}\\
b_{ho}
\end{pmatrix}$,  $W_i = \begin{pmatrix}
W_{ii}\\
W_{if}\\
W_{ig}\\
W_{io}
\end{pmatrix}$ and $b_i = \begin{pmatrix}
b_{ii}\\
b_{if}\\
b_{ig}\\
b_{io}
\end{pmatrix}$.

Convince yourself that, after chunking with [torch.chunk](https://pytorch.org/docs/stable/torch.html?highlight=chunk#torch.chunk), the output of those two linear transformations is equivalent to the output of the eight linear transformations in the LSTM cell calculations above.

#### LSTM Classifier

Having an LSTM cell is not enough: we still need some code that calls it repeatedly, and then makes a prediction from the final hidden state.
You will find that code below. Make sure that you understand it.

In [56]:
class LSTMClassifier(nn.Module):
  """Encodes sentence with an LSTM and projects final hidden state"""

  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, vocab):
    super(LSTMClassifier, self).__init__()
    self.vocab = vocab
    self.hidden_dim = hidden_dim
    self.embed = nn.Embedding(vocab_size, embedding_dim, padding_idx=1)
    self.rnn = MyLSTMCell(embedding_dim, hidden_dim)

    self.output_layer = nn.Sequential(
        nn.Dropout(p=0.5),  # explained later
        nn.Linear(hidden_dim, output_dim)
    )

  def forward(self, x):

    B = x.size(0)  # batch size (this is 1 for now, i.e. 1 single example)
    T = x.size(1)  # timesteps (the number of words in the sentence)

    input_ = self.embed(x)

    # here we create initial hidden states containing zeros
    # we use a trick here so that, if input is on the GPU, then so are hx and cx
    hx = input_.new_zeros(B, self.rnn.hidden_size)
    cx = input_.new_zeros(B, self.rnn.hidden_size)

    # process input sentences one word/timestep at a time
    # input is batch-major (i.e., batch size is the first dimension)
    # so the first word(s) is (are) input_[:, 0]
    outputs = []
    for i in range(T):
      hx, cx = self.rnn(input_[:, i], (hx, cx))
      outputs.append(hx)

    # if we have a single example, our final LSTM state is the last hx
    if B == 1:
      final = hx
    else:
      #
      # This part is explained in next section, ignore this else-block for now.
      #
      # We processed sentences with different lengths, so some of the sentences
      # had already finished and we have been adding padding inputs to hx.
      # We select the final state based on the length of each sentence.

      # two lines below not needed if using LSTM from pytorch
      outputs = torch.stack(outputs, dim=0)           # [T, B, D]
      outputs = outputs.transpose(0, 1).contiguous()  # [B, T, D]

      # to be super-sure we're not accidentally indexing the wrong state
      # we zero out positions that are invalid
      pad_positions = (x == 1).unsqueeze(-1)

      outputs = outputs.contiguous()
      outputs = outputs.masked_fill_(pad_positions, 0.)

      mask = (x != 1)  # true for valid positions [B, T]
      lengths = mask.sum(dim=1)                 # [B, 1]

      indexes = (lengths - 1) + torch.arange(B, device=x.device, dtype=x.dtype) * T
      final = outputs.view(-1, self.hidden_dim)[indexes]  # [B, D]

    # we use the last hidden state to classify the sentence
    logits = self.output_layer(final)
    return logits

#### Dropout

Data sparsity and a small data set can cause *overfitting*. This is a phenomenom that is very likely to occur when training strong and expressive models, like LSTMs, on small data. In practice, if your model overfits, this means that it will be very good at predicting (or "remembering") the sentiment of the training set, but unable to generalise to new, unseen data in the test set. This is undesirable and one technique to mitigate this issue is *dropout*.

A dropout layer is defined by the following formula, which can be applied, for example, to a linear layer:

$$\text{tanh}(W(\mathbf{h}\odot \mathbf{d}) + \mathbf{b})$$

where $\mathbf{d} \in \{0, 1\}^n$, with $d_j \sim \text{Bernoulli}(p)$,

These formula simply means that we *drop* certain parameters during training (by setting them to zero). Which parameters we drop is stochastically determined by a Bernoulli distribution and the probability of each parameter being dropped is set to $p = 0.5$ in our experiments (see the previous cell of code where we define our output layer). A dropout layer can be applied at many different places in our models. This technique helps against the undesirable effect that a model relies on single parameters for prediction (e.g. if $h^{\prime}_j$ is large, always predict positive). If we use dropout, the model needs to learn to rely on different parameters, which is desirable to obtain better generalisation to unseen data.

**Let's train our LSTM!** Note that is will be a lot slower than previous models because we need to do many more computations per sentence.

#### Training

In [ ]:
lstm_args = {
    'vocab_size': len(v.w2i),
    'embedding_dim': 300,
    'hidden_dim': 168,
    'output_dim': len(t2i),
    'vocab': v
}

lstm_no_batch_training_stats = train_with_multiple_seeds(
    model_class=LSTMClassifier,
    model_args=lstm_args,
    seeds=[42, 123, 456],
    pretrained_vectors=vectors,
    freeze_embeddings=True,  # Frozen embeddings
    num_iterations=25000,
    print_every=250,
    eval_every=1000,
    batch_size=1,  # No minibatching
    eval_batch_size=1,
    patience=5,
    max_norm=1.0,
    warmup_steps=100,
    accumulation_steps=4,
    checkpoint_every=1000,
    # Use default batch_fn and prep_fn (not minibatch versions)
    batch_fn=get_examples,
    prep_fn=prepare_example,
    eval_fn=simple_evaluate
)

LSTMClassifier(
  (embed): Embedding(20727, 300, padding_idx=1)
  (rnn): MyLSTMCell(300, 168)
  (output_layer): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=168, out_features=5, bias=True)
  )
)
embed.weight             [20727, 300] requires_grad=False
rnn.weight_ih            [672, 300]   requires_grad=True
rnn.weight_hh            [672, 168]   requires_grad=True
rnn.bias_ih              [672]        requires_grad=True
rnn.bias_hh              [672]        requires_grad=True
output_layer.1.weight    [5, 168]     requires_grad=True
output_layer.1.bias      [5]          requires_grad=True

Total number of parameters: 6534785

Shuffling training data
Iter 250: loss=393.3419, time=1.39s
Iter 500: loss=387.2469, time=2.70s
Iter 750: loss=390.1645, time=4.06s
Iter 1000: loss=378.3716, time=5.44s
iter 1000: dev acc=0.3488
new highscore
Iter 1250: loss=358.7177, time=8.58s
Iter 1500: loss=355.9973, time=9.87s
Iter 1750: loss=361.0908, time=11.15s
Iter 2000: l

/tmp/ipykernel_18179/858537470.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(path)


best model iter 20000: train acc=0.5169, dev acc=0.4523, test acc=0.4706


# Mini-batching



**Why is the LSTM so slow?** Despite our best efforts, we still need to make a lot of matrix multiplications per example (linear in the length of the example) just to get a single classification, and we can only process the 2nd word once we have computed the hidden state for the 1st word (sequential computation).

GPUs are more efficient if we do a few big matrix multiplications, rather than lots of small ones. If we could process multiple examples at the same time, then we could exploit that. That is, we could still process the input sequentially, but doing so for multiple sentences at the same time.

Up to now our "mini-batches" consisted of a single example. This was for a reason: the sentences in our data sets have **different lengths**, and this makes it difficult to process them at the same time.

Consider a batch of 2 sentences:

```
this movie is bad
this movie is super cool !
```

Let's say the IDs for these sentences are:

```
2 3 4 5
2 3 4 6 7 8
```

We cannot feed PyTorch an object with rows of variable length! We need to turn this into a matrix.

The solution is to add **padding values** to our mini-batch:

```
2 3 4 5 1 1
2 3 4 6 7 8
```

Whenever a sentence is shorter than the longest sentence in a mini-batch, we just use a padding value (here: 1) to fill the matrix.

In our computation, we should **ignore** the padding positions (e.g. mask them out) so that paddings do not contribute to the loss.

#### Mini-batch feed
We will now implement a `get_minibatch` function which will replace `get_example` and returns a mini-batch of the requested size.

In [60]:
def get_minibatch(data, batch_size=25, shuffle=True):
  """Return minibatches, optional shuffling"""

  if shuffle:
    print("Shuffling training data")
    random.shuffle(data)  # shuffle training data each epoch

  batch = []

  # yield minibatches
  for example in data:
    batch.append(example)

    if len(batch) == batch_size:
      yield batch
      batch = []

  # in case there is something left
  if len(batch) > 0:
    yield batch

#### Padding function
We will need a function that adds padding 1s to a sequence of IDs so that
it becomes as long as the longest sequence in the minibatch.

In [61]:
def pad(tokens, length, pad_value=1):
  """add padding 1s to a sequence to that it has the desired length"""
  return tokens + [pad_value] * (length - len(tokens))

# example
tokens = [2, 3, 4]
pad(tokens, 5)

[2, 3, 4, 1, 1]

#### New `prepare` function

We will also need a new function that turns a mini-batch into PyTorch tensors.

In [62]:
def prepare_minibatch(mb, vocab):
  """
  Minibatch is a list of examples.
  This function converts words to IDs and returns
  torch tensors to be used as input/targets.
  """
  batch_size = len(mb)
  maxlen = max([len(ex.tokens) for ex in mb])

  # vocab returns 0 if the word is not there
  x = [pad([vocab.w2i.get(t, 0) for t in ex.tokens], maxlen) for ex in mb]

  x = torch.LongTensor(x)
  x = x.to(device)

  y = [ex.label for ex in mb]
  y = torch.LongTensor(y)
  y = y.to(device)

  return x, y

In [63]:
# Let's test our new function.
# This should give us 3 examples.
mb = next(get_minibatch(train_data, batch_size=3, shuffle=False))
for ex in mb:
  print(ex)

Example(tokens=['This', 'is', 'one', 'baaaaaaaaad', 'movie', '.'], tree=Tree('0', [Tree('2', ['This']), Tree('0', [Tree('0', [Tree('2', ['is']), Tree('0', [Tree('2', ['one']), Tree('0', [Tree('0', ['baaaaaaaaad']), Tree('2', ['movie'])])])]), Tree('2', ['.'])])]), label=0, transitions=[0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1])
Example(tokens=['...', 'the', 'plot', 'weaves', 'us', 'into', 'a', 'complex', 'web', '.'], tree=Tree('3', [Tree('2', ['...']), Tree('3', [Tree('2', [Tree('2', ['the']), Tree('2', ['plot'])]), Tree('3', [Tree('3', [Tree('2', [Tree('2', ['weaves']), Tree('3', ['us'])]), Tree('2', [Tree('2', ['into']), Tree('3', [Tree('2', ['a']), Tree('2', [Tree('3', ['complex']), Tree('2', ['web'])])])])]), Tree('2', ['.'])])])]), label=3, transitions=[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1])
Example(tokens=['The', 'power', 'of', 'this', 'script', ',', 'and', 'the', 'performances', 'that', 'come', 'with', 'it', ',', 'is', 'that', 'the', 'whole', 'damned', 'thing', 'did', 

In [64]:
# We should find padding 1s at the end
x, y = prepare_minibatch(mb, v)
print("x", x)
print("y", y)

x tensor([[   74,    11,    51, 20717,   550,     3,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1],
        [   34,     4,  3459, 12005,   136,    98,     8,  1681,   456,     3,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1],
        [   23,   360,     7,    28,  2870,     2,     5,     4,  3939,    16,
           233,    20,    22,     2,    11,    16,     4,   494,  8262,   270,
           123,    41,    84,    70,  3646, 17059,    61,     3]],
       device='cuda:0')
y tensor([0, 3, 2], device='cuda:0')


#### Evaluate (mini-batch version)

We can now update our evaluation function to use mini-batches

In [65]:
def evaluate(model, data,
             batch_fn=get_minibatch, prep_fn=prepare_minibatch,
             batch_size=16):
  """Accuracy of a model on given data set (using mini-batches)"""
  correct = 0
  total = 0
  model.eval()  # disable dropout

  for mb in batch_fn(data, batch_size=batch_size, shuffle=False):
    x, targets = prep_fn(mb, model.vocab)
    with torch.no_grad():
      logits = model(x)

    predictions = logits.argmax(dim=-1).view(-1)

    # add the number of correct predictions to the total correct
    correct += (predictions == targets.view(-1)).sum().item()
    total += targets.size(0)

  return correct, total, correct / float(total)

# LSTM (Mini-batched)

With this, let's run the LSTM again but now using mini-batches!

In [ ]:
lstm_training_stats = train_with_multiple_seeds(
    model_class=LSTMClassifier,
    model_args=lstm_args,
    seeds=[42, 123, 456],
    pretrained_vectors=vectors,
    freeze_embeddings=True,  # Frozen embeddings
    num_iterations=30000,
    print_every=500,
    eval_every=500,
    batch_size=25,
    eval_batch_size=25,
    patience=5,
    max_norm=1.0,
    warmup_steps=100,
    accumulation_steps=4,
    checkpoint_every=1000,
    batch_fn=get_minibatch,
    prep_fn=prepare_minibatch,
    eval_fn=evaluate
)

LSTMClassifier(
  (embed): Embedding(20727, 300, padding_idx=1)
  (rnn): MyLSTMCell(300, 168)
  (output_layer): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=168, out_features=5, bias=True)
  )
)
embed.weight             [20727, 300] requires_grad=False
rnn.weight_ih            [672, 300]   requires_grad=True
rnn.weight_hh            [672, 168]   requires_grad=True
rnn.bias_ih              [672]        requires_grad=True
rnn.bias_hh              [672]        requires_grad=True
output_layer.1.weight    [5, 168]     requires_grad=True
output_layer.1.bias      [5]          requires_grad=True

Total number of parameters: 6534785

Shuffling training data
Iter 250: loss=384.4142, time=2.91s
iter 250: dev acc=0.3252
new highscore
Shuffling training data
Iter 500: loss=341.3048, time=5.92s
iter 500: dev acc=0.4169
new highscore
Shuffling training data
Iter 750: loss=325.3467, time=9.07s
iter 750: dev acc=0.4051
Iter 1000: loss=322.3066, time=12.16s
iter 1000: d

/tmp/ipykernel_18179/858537470.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(path)


best model iter 3750: train acc=0.5640, dev acc=0.4623, test acc=0.4701


Fine-tuning word embeddings

In [ ]:
lstm_finetune_training_stats = train_with_multiple_seeds(
    model_class=LSTMClassifier,
    model_args=lstm_args,  # Same args as above
    seeds=[42, 123, 456],
    pretrained_vectors=vectors,
    freeze_embeddings=False,  # Allow fine-tuning
    num_iterations=30000,
    print_every=500,
    eval_every=500,
    batch_size=25,
    eval_batch_size=25,
    patience=5,
    max_norm=1.0,
    warmup_steps=100,
    accumulation_steps=4,
    checkpoint_every=1000,
    batch_fn=get_minibatch,
    prep_fn=prepare_minibatch,
    eval_fn=evaluate
)

LSTMClassifier(
  (embed): Embedding(20727, 300, padding_idx=1)
  (rnn): MyLSTMCell(300, 168)
  (output_layer): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=168, out_features=5, bias=True)
  )
)
embed.weight             [20727, 300] requires_grad=True
rnn.weight_ih            [672, 300]   requires_grad=True
rnn.weight_hh            [672, 168]   requires_grad=True
rnn.bias_ih              [672]        requires_grad=True
rnn.bias_hh              [672]        requires_grad=True
output_layer.1.weight    [5, 168]     requires_grad=True
output_layer.1.bias      [5]          requires_grad=True

Total number of parameters: 6534785

Shuffling training data
Shuffling training data
Iter 500: loss=716.4662, time=7.61s
Shuffling training data
Iter 1000: loss=607.7030, time=14.88s
iter 1000: dev acc=0.4360
new highscore
Shuffling training data
Shuffling training data
Iter 1500: loss=525.2812, time=22.32s
Shuffling training data
Iter 2000: loss=442.7896, time=29.73s


# Tree-LSTM

In the final part of this lab we will exploit the tree structure of the SST data.
Until now we only used the surface tokens, but remember that our data examples include binary trees with a sentiment score at every node.

In particular, we will implement **N-ary Tree-LSTMs** which are described in:

> Kai Sheng Tai, Richard Socher, and Christopher D. Manning. [Improved Semantic Representations From Tree-Structured Long Short-Term Memory Networks](http://aclweb.org/anthology/P/P15/P15-1150.pdf) ACL 2015.

Since our trees are binary (i.e., N=2), we can refer to these as *Binary Tree-LSTMs*. If you study equations (9) to (14) in the paper, you will find that they are not all too different from the original LSTM that you already have.

You should read this paper carefully and make sure that you understand the approach. You will also find our LSTM baseline there.
Note however that Tree-LSTMs were proposed around the same time by two other groups:

> Phong Le and Willem Zuidema. [Compositional distributional semantics with long short term memory](http://anthology.aclweb.org/S/S15/S15-1002.pdf). *SEM 2015.

> Xiaodan Zhu, Parinaz Sobihani,  and Hongyu Guo. [Long short-term memory over recursive structures](http://proceedings.mlr.press/v37/zhub15.pdf). ICML 2015.

It is good scientific practice to cite all three papers in your report.




## Computation

Do you remember the `transitions_from_treestring` function all the way in the beginning of this lab? Every example contains a **transition sequence** produced by this function. Let's look at it again:



In [ ]:
ex = next(examplereader("trees/dev.txt"))
print(TreePrettyPrinter(ex.tree))
print("Transitions:")
print(ex.transitions)

Note that the tree is **binary**. Every node has two children, except for pre-terminal nodes.

A tree like this can be described by a sequence of **SHIFT (0)** and **REDUCE (1)** actions.

To construct a tree, we can use the transitions as follows:
- **reverse** the sentence (a list of tokens) and call this the **buffer**
   - the first word is now on top (last in the list), and we would get it when calling pop() on the buffer
- create an empty list and call it the **stack**
- iterate through the transition sequence:
  - if it says SHIFT(0), we pop a word from the buffer, and push it to the stack
  - if it says REDUCE(1), we pop the **top two items** from the stack, and combine them (e.g. with a Tree-LSTM!), creating a new node that we push back on the stack
  
Convince yourself that going through the transition sequence above will result in the tree that you see.
For example, we would start by putting the following words on the stack (by shifting 5 times, starting with `It`):

```
Top of the stack:
-----------------
film
lovely
a
's  
It
```
Now we find a REDUCE in the transition sequence, so we get the top two words (film and lovely), and combine them, so our new stack becomes:
```
Top of the stack:
-----------------
lovely film
a
's  
It
```

We will use this approach when encoding sentences with our Tree-LSTM.
Now, our sentence is a reversed list of word embeddings.
When we shift, we move a word embedding to the stack.
When we reduce, we apply the Tree-LSTM to the top two vectors, and the result is a single vector that we put back on the stack.
After going through the whole transition sequence, we will have the root node on our stack! We can use that to classify the sentence.



## Obtaining the transition sequence



So what goes on in the `transitions_from_treestring` function?

The idea ([explained in this blog post](https://devblogs.nvidia.com/recursive-neural-networks-pytorch/)) is that, if we had a tree, we could traverse through the tree, and every time that we find a node containing only a word, we output a SHIFT.
Every time **after** we have finished visiting the children of a node, we output a REDUCE.
(What is this tree traversal called?)

However, our `transitions_from_treestring` function operates directly on the string representation. It works as follows.

We start with the representation:

```
(3 (2 It) (4 (4 (2 's) (4 (3 (2 a) (4 (3 lovely) (2 film))) (3 (2 with) (4 (3 (3 lovely) (2 performances)) (2 (2 by) (2 (2 (2 Buy) (2 and)) (2 Accorsi))))))) (2 .)))
```

First we remove pre-terminal nodes (and add spaces before closing brackets):

```
(3 It (4 (4 's (4 (3 a (4 lovely film ) ) (3 with (4 (3 lovely performances ) (2 by (2 (2 Buy and )  Accorsi ) ) ) ) ) ) . ) )
```

Then we remove node labels:

```
( It ( ( 's ( ( a ( lovely film ) ) ( with ( ( lovely performances) ( by ( ( Buy and )  Accorsi ) ) ) ) ) ) . ) )
```

Then we remove opening brackets:

```
It 's a lovely film ) ) with lovely performances ) by Buy and ) Accorsi ) ) ) ) ) ) . ) )
```

Now we replace words by S (for SHIFT), and closing brackets by R (for REDUCE):

```
S S S S S R R S S S R S S S R S R R R R R R S R R
0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 1 1 1 1 1 1 0 1 1
```

Et voilà. We just obtained the transition sequence!

In [ ]:
# for comparison
seq = ex.transitions
s = " ".join(["S" if t == 0 else "R" for t in seq])
print(s)
print(" ".join(map(str, seq)))

## Coding the Tree-LSTM

The code below contains a Binary Tree-LSTM cell.
It is used in the TreeLSTM class below it, which in turn is used in the TreeLSTMClassifier.
The job of the TreeLSTM class is to encode a complete sentence and return the root node.
The job of the TreeLSTMCell is to return a new state when provided with two children (a reduce action). By repeatedly calling the TreeLSTMCell, the TreeLSTM will encode a sentence. This can be done for multiple sentences at the same time.


#### Exercise
Check the `forward` function and complete the Tree-LSTM formulas.
You can see that we defined a large linear layer for you, that projects the *concatenation* of the left and right child into the input gate, left forget gate, right forget gate, candidate, and output gate.

In [ ]:
class TreeLSTMCell(nn.Module):
  """A Binary Tree LSTM cell"""

  def __init__(self, input_size, hidden_size, bias=True):
    """Creates the weights for this LSTM"""
    super(TreeLSTMCell, self).__init__()

    self.input_size = input_size
    self.hidden_size = hidden_size
    self.bias = bias

    self.reduce_layer = nn.Linear(2 * hidden_size, 5 * hidden_size)
    self.dropout_layer = nn.Dropout(p=0.25)

    self.reset_parameters()

  def reset_parameters(self):
    """This is PyTorch's default initialization method"""
    stdv = 1.0 / math.sqrt(self.hidden_size)
    for weight in self.parameters():
      weight.data.uniform_(-stdv, stdv)

  def forward(self, hx_l, hx_r, mask=None):
    """
    hx_l is ((batch, hidden_size), (batch, hidden_size))
    hx_r is ((batch, hidden_size), (batch, hidden_size))
    """
    prev_h_l, prev_c_l = hx_l  # left child
    prev_h_r, prev_c_r = hx_r  # right child

    B = prev_h_l.size(0)

    # we concatenate the left and right children
    # you can also project from them separately and then sum
    children = torch.cat([prev_h_l, prev_h_r], dim=1)

    # project the combined children into a 5D tensor for i,fl,fr,g,o
    # this is done for speed, and you could also do it separately
    proj = self.reduce_layer(children)  # shape: B x 5D

    # each shape: B x D
    i, f_l, f_r, g, o = torch.chunk(proj, 5, dim=-1)

    # main Tree LSTM computation

    # YOUR CODE HERE
    # You only need to complete the commented lines below.
    # raise NotImplementedError("Implement this.")

    # The shape of each of these is [batch_size, hidden_size]

    i = torch.sigmoid(i)
    f_l = torch.sigmoid(f_l)
    f_r = torch.sigmoid(f_r)
    g = torch.tanh(g)
    o = torch.sigmoid(o)

    c = f_l * prev_c_l + f_r * prev_c_r + i * g
    h = o * torch.tanh(c)

    return h, c

  def __repr__(self):
    return "{}({:d}, {:d})".format(
        self.__class__.__name__, self.input_size, self.hidden_size)

## Explanation of the TreeLSTM class


The code below contains the TreeLSTM class, which implements everything we need in order to encode a sentence from word embeddings. The calculations are the same as in the paper, implemented such that the class `TreeLSTMCell` above is as general as possible and only takes two children to reduce them into a parent.


**Initialize $\mathbf{h}$ and $\mathbf{c}$ outside of the cell for the leaves**

At the leaves of each tree the children nodes are **empty**, whereas in higher levels the nodes are binary tree nodes that *do* have a left and right child (but no input $x$). By initializing the leaf nodes outside of the cell class (`TreeLSTMCell`), we avoid if-else statements in the forward pass.

The `TreeLSTM` class (among other things) pre-calculates an initial $h$ and $c$ for every word in the sentence. Since the initial left and right child are 0, the only calculations we need to do are based on $x$, and we can drop the forget gate calculation (`prev_c_l` and `prev_c_r` are zero). The calculations we do in order to initalize $h$ and $c$ are then:

$$
c_1 =  W^{(u)}x_1 \\
o_1 = \sigma (W^{(i)}x_1) \\
h_1 = o_1 \odot \text{tanh}(c_1)$$
*NB: note that these equations are chosen as initializations of $c$ and $h$, other initializations are possible and might work equally well.*

**Sentence Representations**

All our leaf nodes are now initialized, so we can start processing the sentence in its tree form. Each sentence is represented by a buffer (initially a list with a concatenation of $[h_1, c_1]$ for every word in the reversed sentence), a stack (initially an empty list) and a transition sequence. To encode our sentence, we construct the tree from its transition sequence as explained earlier.

*A short example that constructs a tree:*

We loop over the time dimension of the batched transition sequences (i.e. row by row), which contain values of 0's, 1's and 2's (representing SHIFT, REDUCE and padding respectively). If we have a batch of size 2 where the first example has a transition sequence given by [0, 0, 1, 0, 0, 0, 1] and the second by [0, 0, 1, 0, 0, 1], our transition batch will be given by the following two-dimensional numpy array:

$$
\text{transitions} =
\begin{pmatrix}
0 & 0\\
0 & 0\\
1 & 1\\
0 & 0\\
0 & 0\\
0 & 1\\
1 & 2
\end{pmatrix}
$$
The inner loop (`for transition, buffer, stack in zip(t_batch, buffers, stacks)`) goes over each example in the batch and updates its buffer and stack. The nested loop for this example will then do roughy the following:

```
Time = 0:  t_batch = [0, 0], the inner loop performs 2 SHIFTs.

Time = 1:  t_batch = [0, 0], ".."

Time = 2:  t_batch = [1, 1], causing the inner loop to fill the list child_l and child_r for both examples in the batch. Now the statement if child_l will return True, triggering a REDUCE action to be performed by our Tree LSTM cell with a batch size of 2.

Time = 3:  t_batch = [0, 0], "..".

Time = 4:  t_batch = [0, 0], ".."

Time = 5:  t_batch = [0, 1], one SHIFT will be done and another REDUCE action will be performed by our Tree LSTM, this time of batch size 1.  

Time = 6:  t_batch = [1, 2], triggering another REDUCE action with batch size 1.
```
*NB: note that this was an artificial example for the purpose of demonstrating parts of the code, the transition sequences do not necessarily represent actual trees.*

**Batching and Unbatching**

Within the body of the outer loop over time, we use the functions for batching and unbatching.

*Batching*

Before passing two lists of children to the reduce layer (an instance of `TreeLSTMCell`), we batch the children as they are at this point a list of tensors of variable length based on how many REDUCE actions there are to perform at a certain time step across the batch (let's call the length `L`). To do an efficient forward pass we want to transform the list to a pair of tensors of shape `([L, D], [L, D])`, which the function `batch` achieves.

*Unbatching*

In the same line where we batched the children, we unbatch the output of the forward pass to become a list of states of length `L` again. We do this because we need to loop over each example's transition at the current time step and push the children that are reduced into a parent to the stack.

*The batch and unbatch functions let us switch between the "PyTorch world" (Tensors) and the Python world (easy to manipulate lists).*


In [ ]:
# Helper functions for batching and unbatching states
# For speed we want to combine computations by batching, but
# for processing logic we want to turn the output into lists again
# to easily manipulate.

def batch(states):
  """
  Turns a list of states into a single tensor for fast processing.
  This function also chunks (splits) each state into a (h, c) pair"""
  return torch.cat(states, 0).chunk(2, 1)

def unbatch(state):
  """
  Turns a tensor back into a list of states.
  First, (h, c) are merged into a single state.
  Then the result is split into a list of sentences.
  """
  return torch.split(torch.cat(state, 1), 1, 0)

Take some time to understand the class below, having read the explanation above.

In [ ]:
class TreeLSTM(nn.Module):
  """Encodes a sentence using a TreeLSTMCell"""

  def __init__(self, input_size, hidden_size, bias=True):
    """Creates the weights for this LSTM"""
    super(TreeLSTM, self).__init__()

    self.input_size = input_size
    self.hidden_size = hidden_size
    self.bias = bias
    self.reduce = TreeLSTMCell(input_size, hidden_size)

    # project word to initial c
    self.proj_x = nn.Linear(input_size, hidden_size)
    self.proj_x_gate = nn.Linear(input_size, hidden_size)

    self.buffers_dropout = nn.Dropout(p=0.5)

  def forward(self, x, transitions):
    """
    WARNING: assuming x is reversed!
    :param x: word embeddings [B, T, E]
    :param transitions: [2T-1, B]
    :return: root states
    """

    B = x.size(0)  # batch size
    T = x.size(1)  # time

    # compute an initial c and h for each word
    # Note: this corresponds to input x in the Tai et al. Tree LSTM paper.
    # We do not handle input x in the TreeLSTMCell itself.
    buffers_c = self.proj_x(x)
    buffers_h = buffers_c.tanh()
    buffers_h_gate = self.proj_x_gate(x).sigmoid()
    buffers_h = buffers_h_gate * buffers_h

    # concatenate h and c for each word
    buffers = torch.cat([buffers_h, buffers_c], dim=-1)

    D = buffers.size(-1) // 2

    # we turn buffers into a list of stacks (1 stack for each sentence)
    # first we split buffers so that it is a list of sentences (length B)
    # then we split each sentence to be a list of word vectors
    buffers = buffers.split(1, dim=0)  # Bx[T, 2D]
    buffers = [list(b.squeeze(0).split(1, dim=0)) for b in buffers]  # BxTx[2D]

    # create B empty stacks
    stacks = [[] for _ in buffers]

    # t_batch holds 1 transition for each sentence
    for t_batch in transitions:

      child_l = []  # contains the left child for each sentence with reduce action
      child_r = []  # contains the corresponding right child

      # iterate over sentences in the batch
      # each has a transition t, a buffer and a stack
      for transition, buffer, stack in zip(t_batch, buffers, stacks):
        if transition == SHIFT:
          stack.append(buffer.pop())
        elif transition == REDUCE:
          assert len(stack) >= 2, \
            "Stack too small! Should not happen with valid transition sequences"
          child_r.append(stack.pop())  # right child is on top
          child_l.append(stack.pop())

      # if there are sentences with reduce transition, perform them batched
      if child_l:
        reduced = iter(unbatch(self.reduce(batch(child_l), batch(child_r))))
        for transition, stack in zip(t_batch, stacks):
          if transition == REDUCE:
            stack.append(next(reduced))

    final = [stack.pop().chunk(2, -1)[0] for stack in stacks]
    final = torch.cat(final, dim=0)  # tensor [B, D]

    return final

Just like the LSTM before, we will need an extra class that does the classifications.

In [ ]:
class TreeLSTMClassifier(nn.Module):
  """Encodes sentence with a TreeLSTM and projects final hidden state"""

  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, vocab):
    super(TreeLSTMClassifier, self).__init__()
    self.vocab = vocab
    self.hidden_dim = hidden_dim
    self.embed = nn.Embedding(vocab_size, embedding_dim, padding_idx=1)
    self.treelstm = TreeLSTM(embedding_dim, hidden_dim)
    self.output_layer = nn.Sequential(
        nn.Dropout(p=0.5),
        nn.Linear(hidden_dim, output_dim, bias=True)
    )

  def forward(self, x):

    # x is a pair here of words and transitions; we unpack it here.
    # x is batch-major: [B, T], transitions is time major [2T-1, B]
    x, transitions = x
    emb = self.embed(x)

    # we use the root/top state of the Tree LSTM to classify the sentence
    root_states = self.treelstm(emb, transitions)

    # we use the last hidden state to classify the sentence
    logits = self.output_layer(root_states)
    return logits

## Special `prepare` function for Tree-LSTM

We need yet another `prepare` function. For our implementation, sentences need to be *reversed*. We will do that here.

In [ ]:
def prepare_treelstm_minibatch(mb, vocab):
  """
  Returns sentences reversed (last word first)
  Returns transitions together with the sentences.
  """
  batch_size = len(mb)
  maxlen = max([len(ex.tokens) for ex in mb])

  # vocab returns 0 if the word is not there
  # NOTE: reversed sequence!
  x = [pad([vocab.w2i.get(t, 0) for t in ex.tokens], maxlen)[::-1] for ex in mb]

  x = torch.LongTensor(x)
  x = x.to(device)

  y = [ex.label for ex in mb]
  y = torch.LongTensor(y)
  y = y.to(device)

  maxlen_t = max([len(ex.transitions) for ex in mb])
  transitions = [pad(ex.transitions, maxlen_t, pad_value=2) for ex in mb]
  transitions = np.array(transitions)
  transitions = transitions.T  # time-major

  return (x, transitions), y

## Training

In [ ]:
tree_lstm_args = {
    'vocab_size': len(v.w2i),
    'embedding_dim': 300,
    'hidden_dim': 150,
    'output_dim': len(t2i),
    'vocab': v
}

tree_lstm_training_stats = train_with_multiple_seeds(
    model_class=TreeLSTMClassifier,
    model_args=tree_lstm_args,
    seeds=[42, 123, 456],
    pretrained_vectors=vectors,
    freeze_embeddings=True,  # Frozen embeddings
    num_iterations=30000,
    print_every=500,
    eval_every=500,
    batch_size=25,
    eval_batch_size=25,
    patience=5,
    max_norm=1.0,
    warmup_steps=100,
    accumulation_steps=4,
    checkpoint_every=1000,
    batch_fn=get_minibatch,
    prep_fn=prepare_treelstm_minibatch,  # Special prep function for TreeLSTM
    eval_fn=evaluate
)

In [ ]:
# plot
models_comparison = {
    'BOW': bow_training_stats,
    'CBOW': cbow_training_stats,
    'DeepCBOW': deepcbow_training_stats,
    'PT-DeepCBOW': pt_deep_cbow_training_stats,
    'LSTM-no-batch': lstm_no_batch_training_stats,
    'LSTM-minibatch': lstm_training_stats,
    'LSTM-finetune': lstm_finetune_training_stats,
    'TreeLSTM': tree_lstm_training_stats
}

plot_model_comparison(
    training_stats_list=list(models_comparison.values()),
    model_names=list(models_comparison.keys())
)

# Further experiments and report

For your report, you are expected to answer research questions by doing further experiments.

## Research Questions

Make sure you cover at least the following:

- How important is word order for this task?
- Does the tree structure help to get a better accuracy?
- How does performance depend on the sentence length? Compare the various models. Is there a model that does better on longer sentences? If so, why?
- Do you get better performance if you supervise the sentiment **at each node in the tree**? You can extract more training examples by treating every node in each tree as a separate tree. You will need to write a function that extracts all subtrees given a treestring.
    - Warning: NLTK's Tree function seems to result in invalid trees in some cases, so be careful if you want to parse the string to a tree structure before extraction the phrases.

**To be able to obtain a full grade (10), you should conduct further investigations.** For example, you can also investigate one the following:

- When making a wrong prediction, can you figure out at what point in the tree (sentence) the model fails? You can make a prediction at each node to investigate.
- How does N-ary Tree LSTM compare to the Child-Sum Tree LSTM?
- How do the Tai et al. Tree LSTMs compare to Le & Zuidema's formulation?
- Or... your own research question!

These examples should give you an idea of the expected scope of your further investigations. The maximum number of points awarded for these further investigations depends on the depth of your analysis, and can also be awarded for one of the further research questions already listed above.

In general:

- ***When you report numbers, please report the mean accuracy across 3 (or more) runs with different random seed, together with the standard deviation.*** This is because the final performance may vary per random seed.
More precisely, you should run each model with 3 different seeds, and for each of these 3 runs, evaluate the best model (according to the validation) on the test dataset. The validation dataset is used for finding the best model over iterations, but the accuracy you report should be on the test dataset.

## Report instructions

Your report needs to be written in LaTeX. You are required to use the ACL 2023 template which you can download from or edit directly on [Overleaf](https://www.overleaf.com/latex/templates/acl-2023-proceedings-template/qjdgcrdwcnwp). Make sure your names and student numbers are visible at the top. (Tip: you need to uncomment `\aclfinalcopy`).
You can find some general tips about writing a research paper [here](https://www.microsoft.com/en-us/research/academic-program/write-great-research-paper/), but note that you need to make your own judgment about what is appropriate for this project.

We expect you to use the following structure:
1. Introduction (~1 page) - describe the problem, your research questions and goals, a summary of your findings and contributions. Please cite related work (models, data set) as part of your introduction here, since this is a short paper.
    - Introduce the task and the main goal
    - Present your research questions
    - Motivate the importance of the questions
    - Present and explain your expectations
    - Make clear whether the research questions are addressed or not addressed in the literature
    - Describe the approach you use to answer the research questions
    - Summarise your findings
2. Background (~1/2-1 page) -
cover the main techniques ("building blocks") used in your project (e.g. word embeddings, LSTM, Tree-LSTM) and intuitions behind them. Be accurate and concise.
    - How does each technique work? (Don't just copy the formulas)
    - What is the relation between the techniques?
3. Models (~1/2 page) - Describe the architecture of the final models.
    - How do you use LSTM or Tree-LSTM for the sentiment classification task?
    - What layers do you have, how do you do classification?
    - What is your loss function?
    - Etc.
4. Experiments (~1/2 page) - Describe your experimental setup. This section should allow someone else to reproduce your experiments. Describe how you evaluate the models.
    - Task and the data
    - Training (model, data, parameters and hyper parameters, training algorithms, supervision signals, etc.)
    - Evaluation (e.g. metrics)
5. Results and Analysis (~1 page). Present the results and analyse your findings.
    - Answer each of the research questions you raised in the introduction.
    - Use figures and tables to highlight interesting patterns
    - What are the factors that make model A better than model B in task C? Investigate to prove their effect!
6. Conclusion (~1/4 page). Present the conclusions that can be drawn from your experiments.
    - What have you learned from you experiments?
    - How do your findings relate to what is already known in the literature?
    - Were the results as expected? Any surprising results? Why?
    - Based on what you learned, what would you suggest doing next?


General Tips:

- Math notation – define each variable (either in running text, or in a pseudo-legenda after or before the equation).
- Define technical terminology you need.
- Avoid colloquial language – everything can be said in a scientific-sounding way.
- Avoid lengthy sentences, stay to the point.
- Do not spend space on "obvious" things.
- Do not go over the page limit. (We will deduct points for that.)
- The page limit is 4 pages excluding references and appendix. This is a strict limit; points will be deducted for longer reports.
- There is no strict limit to references and appendix. However, the report needs to remain fully self-contained: the appendix should only include content that is not necessary to understand your work. For example, preprocessing decisions, model parameters, pseudocode, sample system inputs/outputs, and other details that are necessary for the exact replication of your work can be put into the appendix.


An ideal report:
- Precise, scientific, technical, to the point
  - Little general “waffle”/chit-chat
- Not boring – because you don’t explain obvious things too much
- Efficient delivery of (only) the facts that we need to know to understand/reimplement
- Results visually well-presented and described with the correct priority of importance of sub-results
- Insightful analysis – speculation should connect to something interesting and not be too much; the reader “learns something new”
- No typos, no colloquialisms – well-considered language
- This normally means several re-draftings (re-orderings of information)
